In [1]:
#import necessary libraries
import numpy as np
import pandas as pd
from data_loader import load_names_from_web
import altair as alt


In [2]:
dfraw = load_names_from_web(category='national', hide_pre_1937=True, use_existing_files=True)

first_known_year = dfraw['year'].min()
last_known_year = dfraw['year'].max()

#dfraw = dfraw[dfraw['name'] == 'Sarah']

all_years = dfraw.copy()

In [3]:
dfraw

,state,year,name,M/F,count
339406,US,1937,Mary,F,55643
339407,US,1937,Barbara,F,34916
339408,US,1937,Patricia,F,26838
339409,US,1937,Shirley,F,26823
339410,US,1937,Betty,F,25331
...,...,...,...,...,...
2085153,US,2022,Zuberi,M,5
2085154,US,2022,Zydn,M,5
2085155,US,2022,Zylon,M,5
2085156,US,2022,Zymeer,M,5


In [4]:
data = pd.DataFrame()

for year in range(first_known_year, last_known_year+1):

    df = all_years.copy()
    df = df[df['year'] <= year]
    df = df.sort_values(by='year')
    df['cumsum'] = df.groupby(['name', 'M/F'])['count'].cumsum()
    df['sum'] = df.groupby(['name', 'M/F'])['count'].transform('sum')
    medians = df[df['cumsum'] >= df['sum']/2]
    medians = medians.drop_duplicates(subset=['name', 'M/F'], keep='first')
    medians['age'] = year - medians['year']
    thisyear = df[df['year'] == year][['name', 'M/F', 'count']].rename(columns={'count': 'thisyear_count'})
    df = medians.merge(thisyear, how='left', on=['name', 'M/F'])
    df['thisyear_count'] = df['thisyear_count'].fillna(0)
    df = df[['state', 'name', 'M/F', 'sum', 'age', 'thisyear_count']]
    df['year'] = year
    # display(df)
    data = pd.concat([data, df])

data

,state,name,M/F,sum,age,thisyear_count,year
0,US,Mary,F,55643,0,55643.0,1937
1,US,Josh,M,38,0,38.0,1937
2,US,Joey,M,38,0,38.0,1937
3,US,Jared,M,38,0,38.0,1937
4,US,Jame,M,38,0,38.0,1937
...,...,...,...,...,...,...,...
108767,US,Elvi,F,9,0,9.0,2022
108768,US,Bravely,F,14,0,9.0,2022
108769,US,Islagrace,F,14,0,9.0,2022
108770,US,Gwynnevere,F,9,0,9.0,2022


In [5]:
chart = alt.Chart(data).mark_circle().encode(
    x='age',
    y='thisyear_count',
    size='sum',
    order='year',
    tooltip=['name', 'M/F', 'year', 'age', 'thisyear_count', 'sum'],
    color=alt.condition(alt.datum['M/F'] == 'F', alt.value('#FF69B4'), alt.value('#85C1E9'))
).properties(
    width=450,
    height=400
)
chart

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000). For information on how to plot larger datasets in Altair, see the documentation

alt.Chart(...)